In [ ]:
from __future__ import annotations

from emmet.core.tasks import TaskDoc
from monty.serialization import loadfn
import os

from pymatgen.io.validation import ValidationDoc
from pymatgen.io.validation.check_potcar import CheckPotcar

from pymatgen.io.vasp import PotcarSingle, Potcar

In [ ]:
"""
For copyright reasons, the POTCAR for these calculations cannot be distributed with this file, but its summary stats can.
If you have the POTCAR resources set up in pymatgen, you can regenerate the POTCARs used here by enabling `regen_potcars`
"""

regen_potcars = True

def get_potcar_from_spec(potcar_spec : dict) -> Potcar | None:
    potcar_checker = CheckPotcar()
    
    for functional in PotcarSingle._potcar_summary_stats:

        potcar = Potcar()
        matched = [False for _ in range(len(potcar_spec))]
        for ispec, spec in enumerate(potcar_spec):
            titel = spec.get("titel","")
            titel_no_spc = titel.replace(" ","")
            symbol = titel.split(" ")[1].strip()
                
            for stats in PotcarSingle._potcar_summary_stats[functional].get(titel_no_spc,[]):
                
                if potcar_checker.compare_potcar_stats(spec["summary_stats"], stats):
                    potcar.append(PotcarSingle.from_symbol_and_functional(symbol=symbol, functional=functional))
                    matched[ispec] = True
                    break
                    
            if all(matched):
                return potcar
    
def check_calc(calc_dir : str) -> ValidationDoc:
    potcar_filename = None
    if regen_potcars:
        potcar = get_potcar_from_spec(loadfn(os.path.join(calc_dir,"POTCAR.spec.gz")))
        if potcar:
            potcar_filename = os.path.join(calc_dir,"POTCAR.gz")
            potcar.write_file(potcar_filename)
    
    valid_doc = ValidationDoc.from_directory(calc_dir, check_potcar=(regen_potcars and potcar))
    
    if potcar_filename and potcar:
        os.remove(potcar_filename)
        
    return valid_doc
    

In [3]:
"""
An example of an MP-compatible r2SCAN static calculation for GaAs is located in the `MP_compliant` directory.
"""
mp_compliant_doc = check_calc("MP_compliant")
print(mp_compliant_doc.valid)

True


In [ ]:
"""
TaskDocs for these calculations (generated with atomate2) are also saved in these directories.
You can load in the TaskDocs like so:
"""
compliant_task_doc = TaskDoc(
    **loadfn(os.path.join("MP_compliant","MP_compatible_GaAs_r2SCAN_static.json.gz"))
)
mp_compliant_doc = ValidationDoc.from_task_doc(compliant_task_doc)
print(mp_compliant_doc.valid)

True


In [7]:
"""
An example of an MP incompatible r2SCAN static calculation for GaAs is located in the `MP_non_compliant` directory.

This calculation uses a lower ENCUT, ENAUG, and k-point density (larger KSPACING) than is permitted by the appropriate input set, `pymatgen.io.vasp.sets.MPScanStaticSet`.
These reasons are reflected transparently in the output reasons.
"""
mp_non_compliant_doc = check_calc("MP_non_compliant")
print(mp_non_compliant_doc.valid)
for reason in mp_non_compliant_doc.reasons:
    print(reason)

non_compliant_task_doc = TaskDoc(
    **loadfn(os.path.join("MP_non_compliant","MP_incompatible_GaAs_r2SCAN_static.json.gz"))
)
mp_non_compliant_doc_from_taskdoc = ValidationDoc.from_task_doc(non_compliant_task_doc)
print(mp_non_compliant_doc_from_taskdoc.valid)
print(mp_non_compliant_doc_from_taskdoc.reasons == mp_non_compliant_doc_from_taskdoc.reasons)

False
INPUT SETTINGS --> KPOINTS or KSPACING: 64 kpoints were used, but it should have been at least 194.
INPUT SETTINGS --> ENAUG: is 900.0, but should be >= 1360. 
INPUT SETTINGS --> ENCUT: is 450.0, but should be >= 680. 
False
True
